In [49]:
import pandas as pd
import numpy as np
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer as SIA
from pprint import pprint
import praw

In [51]:
reddit=praw.Reddit(client_id='Vuj-VpxhcJRzBg',
                  client_secret='EtCSFClXSR-mULqW1_qWMAnKHww',
                  user_agent='EtCSFClXSR-mULqW1_qWMAnKHww')

In [52]:
headlines=set()
for sub in reddit.subreddit('politics').new(limit=None):
    headlines.add(sub.title)
print(len(headlines))

929


In [53]:
sia = SIA()
results=[]
for line in headlines:
    scores = sia.polarity_scores(line)
    scores['headlines']=line
    results.append(scores)
df = pd.DataFrame.from_records(results)
df.head()

,neg,neu,pos,compound,headlines
0,0.00,1.000,0.000,0.0000,How is presidential debate moderator Kristen W...
1,0.00,0.753,0.247,0.3182,NYPD says union's Trump endorsement won't affe...
2,0.20,0.800,0.000,-0.3612,Trump struggles to lift glass of water during ...
3,0.07,0.930,0.000,-0.3400,Trump may fire Bill Barr and FBI director for ...
4,0.00,0.752,0.248,0.3182,Top 3 Legal Questions About Rudy Giuliani’s ‘B...


In [54]:
df['compound'].value_counts()

 0.0000    366
-0.4019     21
 0.4019     20
 0.3182     19
 0.5574     19
          ... 
 0.2716      1
-0.7430      1
-0.8720      1
 0.2484      1
-0.7269      1
Name: compound, Length: 167, dtype: int64

In [55]:
df['comp_score'] = df['compound'].apply(lambda score: 'pos' if score>0 else ('neu' if score==0 else 'neg'))

In [56]:
df['comp_score'].value_counts()

neu    366
neg    324
pos    239
Name: comp_score, dtype: int64

In [57]:
df.head()

,neg,neu,pos,compound,headlines,comp_score
0,0.00,1.000,0.000,0.0000,How is presidential debate moderator Kristen W...,neu
1,0.00,0.753,0.247,0.3182,NYPD says union's Trump endorsement won't affe...,pos
2,0.20,0.800,0.000,-0.3612,Trump struggles to lift glass of water during ...,neg
3,0.07,0.930,0.000,-0.3400,Trump may fire Bill Barr and FBI director for ...,neg
4,0.00,0.752,0.248,0.3182,Top 3 Legal Questions About Rudy Giuliani’s ‘B...,pos


In [58]:
a = "This is a good movie"
sia.polarity_scores(a)

{'neg': 0.0, 'neu': 0.508, 'pos': 0.492, 'compound': 0.4404}

In [59]:
from sklearn.metrics import accuracy_score,classification_report,confusion_matrix

In [60]:
df['label']=0
df.loc[df['compound']>0.1,'label']=1
df.loc[df['compound']<-0.1,'label']= -1
df.head()

,neg,neu,pos,compound,headlines,comp_score,label
0,0.00,1.000,0.000,0.0000,How is presidential debate moderator Kristen W...,neu,0
1,0.00,0.753,0.247,0.3182,NYPD says union's Trump endorsement won't affe...,pos,1
2,0.20,0.800,0.000,-0.3612,Trump struggles to lift glass of water during ...,neg,-1
3,0.07,0.930,0.000,-0.3400,Trump may fire Bill Barr and FBI director for ...,neg,-1
4,0.00,0.752,0.248,0.3182,Top 3 Legal Questions About Rudy Giuliani’s ‘B...,pos,1


In [61]:
print("Positive headlines:\n")
pprint(list(df[df['label']==1].headlines)[:10],width=100)

print("\nNegative headlines:\n")
pprint(list(df[df['label']==-1].headlines)[:10],width=100)

Positive headlines:

["NYPD says union's Trump endorsement won't affect enforcement",
 'Top 3 Legal Questions About Rudy Giuliani’s ‘Beyond Cringe’ Scene in ‘Borat,’ Answered',
 'Trump’s tax returns: Why the Supreme Court should end things now',
 'Federal Judges Claim It’s Unconstitutional for States to Protect Voting Rights | The federal '
 'judiciary is on the brink of mandating voter suppression—even in states that don’t want it.',
 "No need to feel down: Trump's 'YMCA' rally dancing is campaign bright spot",
 'Powerful video of Biden comforting son of school shooting victim resurfaces',
 'How anti-vaxxers have rebranded in the coronavirus era: ‘Our bodies are ours, not for someone '
 "else to govern' | Vaccine opponents have long been undergoing a rebranding effort to shift their "
 'messaging toward the promotion of civil liberties',
 'McConnell helps ‘working families’ by killing coronavirus relief. That’s rich, Mitch.',
 'McConnell says nothing wrong with health after photos sho